In [41]:
import numpy as np
import h5py 
from typing import Tuple
from base_reader import BaseReader
import yaml
import pprint
from convert import *
import json
import xml.etree.ElementTree as ET

In [42]:
with open("NXmpes_core_draft.yml", 'r') as f:
    meta_dict= yaml.full_load(f)
    print(meta_dict)

{'start_time': '2021-12-13T13:45:14+01:00', 'type': 'Laser', 'name': 'Tungsten Diselenide', 'probe': 'X-ray', 'distance': '0.0', 'incident_energy': '21.7', 'incident_energy_spread': '0.110', 'incident_polarization': '[1,1,0,0]', 'energy_resolution': '0.140', 'momentum_resolution': '0.08', 'fast_axes': '[x,y,tof]', 'slow_axes': 'None', 'scheme': 'tof', 'lens_mode': '6kV_kmode_m4.2_30VTOF_453ns_focus_Gated_21.0V_FMCP.sav', 'projection': 'reciprocal', 'pass_energy': '30.0', 'energy_scan_mode': 'fixed', 'amplifier_type': 'MCP', 'detector_type': 'DLD', 'sensor_pixels': '[1800,1800]', 'applied': 'True', 'chemical_formula': 'WSe2', 'description': '2-D semiconductor', 'temperature': '18', 'situation': 'vacuum', 'pressure': '0.001'}


In [43]:
#Writing the metadata to an hdf5 file.
with h5py.File('fixed_metadata.h5','w') as hf:
    hf.attrs.update(meta_dict)

In [44]:
#Generating the template from Sherjeel's code

nxdl= "NXmpes_core_draft.nxdl.xml" #appdef
tree = ET.parse(nxdl)

template = {}
root = get_first_group(tree.getroot())
generate_template_from_nxdl(root, '', template)


In [45]:
class Reader(BaseReader):
    
    def reader(template, input_file): #For the case of one input file
        with h5py.File(input_file, 'r') as f:

            metadata= dict(f.attrs.items())

            for path in template:
                path_trunc= path.split('/')[-1]

                if path_trunc in metadata:
                    template[path]= metadata[path_trunc]
                    
        return template

In [46]:
READER = Reader
pprint.pprint(READER.reader(template, "fixed_metadata.h5"))

{'/NXentry/@entry': None,
 '/NXentry/NXdata/@energy_indices': None,
 '/NXentry/NXdata/@kx_indices': None,
 '/NXentry/NXdata/@signal': None,
 '/NXentry/NXdata/data': None,
 '/NXentry/NXinstrument/NXbeam/distance': '0.0',
 '/NXentry/NXinstrument/NXbeam/distance/units': None,
 '/NXentry/NXinstrument/NXbeam/incident_energy': '21.7',
 '/NXentry/NXinstrument/NXbeam/incident_energy/units': None,
 '/NXentry/NXinstrument/NXbeam/incident_energy_spread': '0.110',
 '/NXentry/NXinstrument/NXbeam/incident_energy_spread/units': None,
 '/NXentry/NXinstrument/NXbeam/incident_polarization': '[1,1,0,0]',
 '/NXentry/NXinstrument/NXbeam/incident_polarization/units': None,
 '/NXentry/NXinstrument/NXelectronanalyser/NXcollectionlens/NXaperture/description': '2-D '
                                                                                     'semiconductor',
 '/NXentry/NXinstrument/NXelectronanalyser/NXcollectionlens/NXaperture/shape': None,
 '/NXentry/NXinstrument/NXelectronanalyser/NXcollectionlens/N